### Try to find a way to compute the synchrony of neurons accross time

In [43]:
import sys
import itertools

import numpy as np
import pandas as pd
import scipy.signal
from scipy.signal import savgol_filter

from tqdm import tqdm

import neuroseries as nts


import time
import matplotlib.pyplot as plt
%matplotlib qt

import seaborn as sns 
import bk.load
import bk.compute
import bk.plot

import os

# import basefunction.anass_load as at

In [44]:
# def binEdges2centers(BinEdges):
#     #Vitor Code (Thanks)
#     BinCenters = np.convolve(BinEdges,[.5,.5],'same')
#     BinCenters = BinCenters[1::]

#     return BinCenters

In [45]:
# def synchrony(neurons,metadata,binSize,stru,sm = 201,method = 'corr',plot = True):
#     #Old version, please refer to below
#     try: bk.load.session
#     except NameError: bk.load.session = None
    
# #     binSize = 0.1
#     winSize = int(1/binSize)
#     step = int(1/binSize)

    
#     print(len(neurons))
#     print(binSize)
    

#     binned,t = binSpikes(neurons,binSize)
    
#     stru_shank = {}
#     stru_mask  = {}
    
#     for s in stru:
#             stru_shank.update({s:at.get_structure_shanks(bk.load.session,s)})
#             if not np.any(stru_shank[s]):
#                 print('Could not compute ' + s + ' because is you don\'t have shank in it')
#                 continue
#             stru_mask.update({s:[i in stru_shank[s] for i in metadata[:,0]]})
    
#     if method == 'corr':
#         pbar = ProgressBar()
#         corr = np.empty((int(binned.shape[1]/step)+1,len(neurons),len(neurons)))   

#         for i,j in enumerate(pbar(range(0,binned.shape[1],step))):
#             corr[i] = np.corrcoef(binned[:,j:j+winSize])
#             corr[i][np.isnan(corr[i])] = 0
#         t = t[::step]
#         # corr = np.abs(corr)

#         stru_corr  = {}
#         stru_sync  = {}
            
#         for s in stru_mask.keys():
#             print('computing '+s)
#             stru_corr.update({s:corr[:,:,stru_mask[s]][:,stru_mask[s]]})
#             stru_sync.update({s:np.nanmean(stru_corr[s],axis = (1,2))})

#         stru_corr.update({'Hpc-BLA':corr[:,:,stru_mask['Hpc']][:,stru_mask['BLA']]})
#         stru_sync.update({'Hpc-BLA':np.nanmean(stru_corr['Hpc-BLA'],axis = (1,2))})

#         # for s,s2 in itertools.combinations(stru_mask.keys(),2):
#         #     print('computing '+s+'-'+s2)
#         #     stru_corr.update({s+'-'+s2:corr[:,:,stru_mask[s]][:,stru_mask[s2]]})
#         #     stru_sync.update({s+'-'+s2:np.nanmean(stru_corr[s+'-'+s2],axis = (1,2))})

#     elif method == 'coactivation':
#         #Binarize the binning.
#         #count number of cell active at each timebin
#         # nb of cell active / total number in da category for each time point. 
    
#         binarize_binned = np.where(binned<1,binned,1)
#         stru_sync  = {}
#         for s in stru_mask.keys():
#             print('computing '+s)
#             stru_sync.update({s:np.sum(binarize_binned[stru_mask[s],:],0)/binarize_binned.shape[0]})
            
#     elif method == 'firing':
#         stru_sync  = {}
#         for s in stru_mask.keys():
#             print('computing '+s)
#             stru_sync.update({s:np.mean(binned[stru_mask[s],:],0)/binSize})
#     else:    

#         print(method + ' Is an unknown method')
#         raise ValueError('Unknow Method')
    
    
#     if plot:
#         for s in stru_sync:
#             plt.plot(t,savgol_filter(stru_sync[s],sm,3))
#         plt.title('synchrony during ' + bk.session)
#         plt.legend([s for s in stru_sync.keys()])
#         plt.ylabel('r')
#         plt.xlabel('Time(s)')

#         Rem = nts.IntervalSet(scipy.io.loadmat('States.mat')['Rem'][:,0],scipy.io.loadmat('States.mat')['Rem'][:,1],time_units = 's')
#         plotIntervals(Rem,'darkorange')

#         sws = nts.IntervalSet(scipy.io.loadmat('States.mat')['sws'][:,0],scipy.io.loadmat('States.mat')['sws'][:,1],time_units = 's')
#         plotIntervals(sws,'grey')
    
#     return stru_sync

# def binSpikes(neurons,binSize = 0.05,start = 0,stop = 0):
#     if stop == 0:
#         stop = np.max([neuron.as_units('s').index[-1] for neuron in neurons])

#     bins = np.arange(start,stop,binSize)
#     binned = []
#     for neuron in neurons:
#         hist,b = np.histogram(neuron.as_units('s').index,bins = bins)
#         binned.append(hist)

#     return np.array(binned), binEdges2centers(b)

# def plotIntervals(intervals,col):
#     for interval in intervals.as_units('s').values:
#         plt.axvspan(interval[0],interval[1], facecolor=col, alpha=0.5)

In [159]:
def synchrony(neurons,binSize,winSize,step,sm = 201,method = 'corr',plot = True):
    try: bk.load.session
    except NameError : bk.load.session = None
    
    winSize = int(winSize / binSize)
    step = int(step/binSize)
    print('Computing synchrony with following parameters')
    print('winSize (bins) : ',winSize*2)
    print('Step    (bins) : ',step)
    print(len(neurons),' neurons')
    t,binned = bk.compute.binSpikes(neurons,binSize)
    
    if method.lower() == 'corr':
        
        #Don't really why, but it's working this way ... 
        corr_sample = binned.shape[1]/step
        if not corr_sample.is_integer(): 
            corr_sample = int(corr_sample)+1
        else:
            corr_sample = int(corr_sample)
            
        corr = np.zeros(corr_sample)
        for i,j in enumerate(tqdm(range(0,binned.shape[1],step))):
            
            corr_mat = np.corrcoef(binned[:,j-winSize:j+winSize])
#             corr_mat[np.isnan(corr_mat)] = 0
            corr[i] = np.nanmean(corr_mat)
        t = t[::step]
        
        corr = nts.Tsd(t,corr,time_units = 's')
        return corr

def average_over_transitions(signal,nBins,intervals,timing,states):
    if len(intervals) < 4: return 0
    
    c_all = []
    pre_all = []
    post_all = []
    
    for event,t in zip(intervals.iloc,timing.index.values):
        pre = nts.IntervalSet(event[0],t)
        post = nts.IntervalSet(t,event[1])
        event = nts.IntervalSet(event[0],event[1])

        if pre.tot_length(time_units = 's') < 150: continue
        if post.tot_length(time_units = 's') < 50: continue
        
        s = signal.restrict(event)
        s.index = s.index-t
        s_pre = signal.restrict(pre)
        s_post = signal.restrict(post)
        
        plt.plot(s.as_units('s')[-150:50],'grey',alpha = 0.1)
        c_all.append(s.as_units('s')[-150:50])
        pre_all.append(np.mean(s.as_units('s')[-150:0].values)) 
        post_all.append(np.mean(s.as_units('s')[0:50].values))
        
    c_all = np.array(c_all)
    c_all_ave = nts.Tsd(s.as_units('s')[-150:50].index.values*1_000_000,np.nanmean(c_all,0))
    
    plt.axvline(0,color = 'red')
    plt.axvspan(0,50,facecolor = 'orange',alpha = 0.5)
    plt.plot(c_all_ave.as_units('s'))
    print(np.nanmean(np.array([pre_all,post_all]),1))
    
    return c_all

In [160]:
def main(path):
    bk.load.current_session(path)
    neurons,metadata = bk.load.loadSpikeData(bk.load.path)
    states = bk.load.states()
    stru = metadata.Region.unique()
    stru = np.delete(stru,stru == 'None')
    
    transitions = bk.compute.transitions_times(states)
    plt.figure()
    plt.suptitle(bk.load.session)
    c_all = {}
    for i,s in enumerate(stru,0):
        t = 'Pyr'
        print(s,t)
        c = synchrony(neurons[(metadata.Region == s) & (metadata.Type == t)],binSize = 0.005,winSize = 1,step = 1)
        c_sm = bk.compute.nts_smooth(c,100,50)
        plt.subplot(len(stru),1,i+1)
        plt.title(f'{s} + {t}')
        average = average_over_transitions(c,[30,12],transitions[0][('sws', 'Rem')],transitions[1][('sws', 'Rem')],states)
        c_all.update({s:average})
    #     plt.plot(c.as_units('s'),alpha = 0.8) 
    #     plt.plot(c_sm.as_units('s'),alpha = 0.9)

        #     c = nts.Tsd(c.index.values,np.convolve(c.values,g,'same'))
    #     plt.plot(bk.compute.nts_smooth(c,100,).as_units('s'))
    # bk.plot.intervals(states['Rem'],'orange')
    # bk.plot.intervals(states['sws'],'grey')
    return c_all

In [71]:
path = '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'
os.chdir(path)
bk.load.path = path
bk.load.rat = 8
bk.load.session = 'Rat08-20130713'
bk.load.day = 6

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/media/billel/GGLab-01-BK/DATA/Rat08-20130713/'

In [105]:
neurons,metadata = bk.load.loadSpikeData(bk.load.path)

Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy


In [109]:
interval = nts.IntervalSet(1250,1251,time_units='s')

In [119]:
for i,n in enumerate(neurons):
    neurons[i] = n.restrict(interval)

In [111]:
t,b = bk.compute.binSpikes(neurons,binSize= 0.050)
corr = np.corrcoef(b)
plt.figure()
plt.imshow(corr)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [72]:
%matplotlib qt

In [161]:
main('Z:\Rat08\Rat08-20130713')

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713
Data already saved in Numpy format, loading them from here:
Rat08-20130713-neurons.npy
Rat08-20130713-metadata.npy
Looking at transition from Rem  to  sws
Looking at transition from Rem  to  wake
Looking at transition from Rem  to  drowsy
Looking at transition from sws  to  Rem
Looking at transition from sws  to  wake
Looking at transition from sws  to  drowsy
Looking at transition from wake  to  Rem
Looking at transition from wake  to  sws
Looking at transition from wake  to  drowsy
Looking at transition from drowsy  to  Rem
Looking at transition from drowsy  to  sws
Looking at transition from drowsy  to  wake
Hpc Pyr
Computing synchrony with following parameters
winSize (bins) :  400
Step    (bins) :  200
55  neurons


  0%|                                                                                        | 0/22680 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: RuntimeWarning: Mean

[0.04405542 0.06505787]
BLA Pyr
Computing synchrony with following parameters
winSize (bins) :  400
Step    (bins) :  200
83  neurons


100%|██████████████████████████████████████████████████████████████████████████| 22680/22680 [00:09<00:00, 2291.64it/s]


[0.03124476 0.03347193]
VEn Pyr
Computing synchrony with following parameters
winSize (bins) :  400
Step    (bins) :  200
19  neurons


100%|██████████████████████████████████████████████████████████████████████████| 22680/22680 [00:04<00:00, 5176.66it/s]


[0.13796508 0.12988148]
Pir Pyr
Computing synchrony with following parameters
winSize (bins) :  400
Step    (bins) :  200
66  neurons


100%|██████████████████████████████████████████████████████████████████████████| 22680/22680 [00:09<00:00, 2461.95it/s]


[0.03383098 0.03102754]


{'Hpc': array([[0.03132757, 0.0362891 , 0.05791865, ..., 0.08062644, 0.05875151,
         0.04548292],
        [0.03810431, 0.04099019, 0.03335246, ..., 0.0429612 , 0.04574816,
         0.04731604],
        [0.03263422, 0.04323061, 0.03202493, ..., 0.04814957, 0.05889714,
         0.06582373],
        ...,
        [0.03093928, 0.03875635, 0.03822057, ..., 0.04038981, 0.04319971,
         0.04378941],
        [0.06511455, 0.03788135, 0.04427912, ..., 0.08089406, 0.06328917,
         0.05203788],
        [0.11917939, 0.1320573 , 0.13574546, ..., 0.04548472, 0.05238341,
         0.0512629 ]]),
 'BLA': array([[0.02930958, 0.02466287, 0.02455832, ..., 0.02677227, 0.02518651,
         0.02762031],
        [0.02789554, 0.03265995, 0.03253286, ..., 0.02112285, 0.02585693,
         0.0289702 ],
        [0.02996874, 0.03028512, 0.02918044, ..., 0.03297709, 0.03105357,
         0.03350639],
        ...,
        [0.0271851 , 0.03149432, 0.04351361, ..., 0.02602291, 0.03690667,
         0.03574352]

In [97]:
bk.load.current_session('Z:\Rat08\Rat08-20130710')
neurons,metadata = bk.load.loadSpikeData(bk.load.path)

neurons = neurons[(metadata.Region == 'CeCM') & (metadata.Type == 'Pyr')]

c = synchrony(neurons,binSize = 0.025,winSize = 5,step = 1.125)

Rat : 8 on day : 3
Working with session Rat08-20130710 @ Z:\Rat08\Rat08-20130710
Data already saved in Numpy format, loading them from here:
Rat08-20130710-neurons.npy
Rat08-20130710-metadata.npy
Computing synchrony with following parameters
winSize (bins) :  80
Step    (bins) :  45
13  neurons


  0%|                                                                                        | 0/22396 [00:00<?, ?it/s]C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_methods.py:163: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2551: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2480: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarnin

22396
22396


In [122]:
bk.load.current_session()

Rat : 8 on day : 6
Working with session Rat08-20130713 @ Z:\Rat08\Rat08-20130713


True

In [101]:
batch = bk.load.batch(main)

 20%|████████████████▏                                                                 | 12/61 [01:54<02:13,  2.72s/it]

Loading Data from Rat08-20130722
Rat : 8 on day : 13
Working with session Rat08-20130722 @ Z:\Rat08\Rat08-20130722
Data already saved in Numpy format, loading them from here:
Rat08-20130722-neurons.npy
Rat08-20130722-metadata.npy
Error in session Rat08-20130722


 20%|████████████████▏                                                                 | 12/61 [01:54<07:46,  9.52s/it]


KeyboardInterrupt: 

In [625]:
%matplotlib inline

In [52]:
a = batch['Rat08-20130708']

KeyError: 'Rat08-20130708'

In [35]:
a = np.array(['test','deux'])

In [37]:
np.delete(a,1)

array(['test'], dtype='<U4')

In [38]:
a

array(['test', 'deux'], dtype='<U4')

In [53]:
for i,p in a.items():
    plt.figure()

    plt.plot(p)
    plt.title(i)

AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [69]:
transitions[1][('Rem', 'sws')]

Time (us)
3887500000    NaN
3969500000    NaN
4023500000    NaN
4567500000    NaN
5488500000    NaN
7374500000    NaN
7787500000    NaN
17363500000   NaN
17496500000   NaN
17851500000   NaN
18131500000   NaN
19186500000   NaN
19545500000   NaN
19817500000   NaN
21358500000   NaN
21674500000   NaN
dtype: float64

In [64]:
for i in transitions[0].items(): print(i[0])

('Rem', 'sws')
('Rem', 'wake')
('Rem', 'drowsy')
('sws', 'Rem')
('sws', 'wake')
('sws', 'drowsy')
('wake', 'Rem')
('wake', 'sws')
('wake', 'drowsy')
('drowsy', 'Rem')
('drowsy', 'sws')
('drowsy', 'wake')


In [40]:
transitions

({('Rem',
   'sws'):           start          end
  0    3842000000   3916000000
  1    3917000000   3992000000
  2    3993000000   4529000000
  3    4530000000   4648000000
  4    5427000000   6069000000
  5    7312000000   7759000000
  6    7760000000   7909000000
  7   17260000000  17429000000
  8   17430000000  17669000000
  9   17811000000  18076000000
  10  18077000000  18528000000
  11  19168000000  19448000000
  12  19449000000  19771000000
  13  19772000000  20363000000
  14  21287000000  21629000000
  15  21630000000  21850000000,
  ('Rem',
   'wake'):         start          end
  0  2430000000   2825000000
  1  6070000000   6367000000
  2  7910000000   8484000000
  3  9334000000  13214000000,
  ('Rem',
   'drowsy'):          start          end
  0   1927000000   2103000000
  1   3276000000   3382000000
  2   4677000000   4863000000
  3   4864000000   5043000000
  4   8798000000   8871000000
  5  16146000000  16435000000
  6  16766000000  16800000000
  7  18529000000  1866200

In [6]:
states.update(transitions[0])

In [7]:
transitions2 = bk.compute.transitions_times(states)

Looking at transition from Rem  to  sws
Looking at transition from Rem  to  wake
Looking at transition from Rem  to  drowsy
Looking at transition from Rem  to  ('Rem', 'sws')
Looking at transition from Rem  to  ('Rem', 'wake')
Looking at transition from Rem  to  ('Rem', 'drowsy')
Looking at transition from Rem  to  ('sws', 'Rem')
Looking at transition from Rem  to  ('sws', 'wake')
Looking at transition from Rem  to  ('sws', 'drowsy')
Looking at transition from Rem  to  ('wake', 'sws')
Looking at transition from Rem  to  ('wake', 'drowsy')
Looking at transition from Rem  to  ('drowsy', 'Rem')
Looking at transition from Rem  to  ('drowsy', 'sws')
Looking at transition from Rem  to  ('drowsy', 'wake')
Looking at transition from sws  to  Rem
Looking at transition from sws  to  wake
Looking at transition from sws  to  drowsy
Looking at transition from sws  to  ('Rem', 'sws')
Looking at transition from sws  to  ('Rem', 'wake')
Looking at transition from sws  to  ('Rem', 'drowsy')
Looking at 

In [8]:
x = plt.subplot(4,1,1)
bk.plot.intervals(states['Rem'],'orange')
bk.plot.intervals(states['sws'],'grey')
plt.subplot(4,1,2,sharex=x)
bk.plot.intervals(states['sws','Rem'])
plt.subplot(4,1,3,sharex=x)
bk.plot.intervals(transitions2[0][('sws','Rem'),'sws'])
plt.subplot(4,1,4,sharex=x)
bk.plot.intervals(transitions2[0]['sws',('Rem','sws')])

In [36]:
bk.plot.intervals(states['sws','Rem'])

In [47]:
for i in transitions2[0]: print(i)

('Rem', 'sws')
('Rem', 'wake')
('Rem', 'drowsy')
('Rem', ('Rem', 'sws'))
('Rem', ('Rem', 'wake'))
('Rem', ('Rem', 'drowsy'))
('Rem', ('sws', 'Rem'))
('Rem', ('sws', 'wake'))
('Rem', ('sws', 'drowsy'))
('Rem', ('wake', 'sws'))
('Rem', ('wake', 'drowsy'))
('Rem', ('drowsy', 'Rem'))
('Rem', ('drowsy', 'sws'))
('Rem', ('drowsy', 'wake'))
('sws', 'Rem')
('sws', 'wake')
('sws', 'drowsy')
('sws', ('Rem', 'sws'))
('sws', ('Rem', 'wake'))
('sws', ('Rem', 'drowsy'))
('sws', ('sws', 'Rem'))
('sws', ('sws', 'wake'))
('sws', ('sws', 'drowsy'))
('sws', ('wake', 'sws'))
('sws', ('wake', 'drowsy'))
('sws', ('drowsy', 'Rem'))
('sws', ('drowsy', 'sws'))
('sws', ('drowsy', 'wake'))
('wake', 'Rem')
('wake', 'sws')
('wake', 'drowsy')
('wake', ('Rem', 'sws'))
('wake', ('Rem', 'wake'))
('wake', ('Rem', 'drowsy'))
('wake', ('sws', 'Rem'))
('wake', ('sws', 'wake'))
('wake', ('sws', 'drowsy'))
('wake', ('wake', 'sws'))
('wake', ('wake', 'drowsy'))
('wake', ('drowsy', 'Rem'))
('wake', ('drowsy', 'sws'))
('wake',

In [23]:
transitions[0]

{('Rem',
  'sws'):           start          end
 0    3842000000   3916000000
 1    3917000000   3992000000
 2    3993000000   4529000000
 3    4530000000   4648000000
 4    5427000000   6069000000
 5    7312000000   7759000000
 6    7760000000   7909000000
 7   17260000000  17429000000
 8   17430000000  17669000000
 9   17811000000  18076000000
 10  18077000000  18528000000
 11  19168000000  19448000000
 12  19449000000  19771000000
 13  19772000000  20363000000
 14  21287000000  21629000000
 15  21630000000  21850000000,
 ('Rem',
  'wake'):         start          end
 0  2430000000   2825000000
 1  6070000000   6367000000
 2  7910000000   8484000000
 3  9334000000  13214000000,
 ('Rem',
  'drowsy'):          start          end
 0   1927000000   2103000000
 1   3276000000   3382000000
 2   4677000000   4863000000
 3   4864000000   5043000000
 4   8798000000   8871000000
 5  16146000000  16435000000
 6  16766000000  16800000000
 7  18529000000  18662000000
 8  21851000000  22406000000,

In [17]:
# path = bk.sessions()['Path'][4]
# bk.current_session(path)
# neurons,metadata = bk.loadSpikeData(bk.path)

methods = ['firing','coactivation','corr']
sms = [5001,5001,201]
plt.figure()
sub = plt.subplot(3,1,1)
for i,m in enumerate(methods):
    plt.subplot(len(methods),1,i+1,sharex = sub)
    syn = synchrony(neurons,metadata,0.05,['Hpc','BLA'],method = m ,sm = sms[i])
    plt.xlabel('')
    if i == 0:
        plt.title('Firing Rates')
    else:
        plt.title('Syncrhony calculated using '+ m )
plt.suptitle(bk.load.session)
plt.show()

252
0.05


TypeError: '(slice(None, None, None), 0)' is an invalid key

,Shank,Id,Region,Type
0,1,2,Hpc,Pyr
1,2,2,Hpc,Pyr
2,2,3,Hpc,Pyr
3,2,4,Hpc,Pyr
4,2,5,Hpc,Pyr
...,...,...,...,...
247,19,2,BLA,Pyr
248,19,3,BLA,Pyr
249,19,4,BLA,Pyr
250,19,5,BLA,Pyr


In [214]:
plt.subplot(3,1,1)
plt.title('Firing Rates')

Text(0.5, 1.0, 'Firing Rates')

In [184]:
methods = ['firing','coactivation','corr']
sms = [11,1001,201]
plt.figure()
sub = plt.subplot(3,1,1)
for i,m in enumerate(methods):
    plt.subplot(len(methods),1,i+1)
    syn = synchrony(neurons,metadata,100,['Hpc','BLA'],method = m ,sm = sms[i])
    plt.xlabel('')
    plt.title('Synchrony calculated using '+ m )
plt.show()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  


40
100
computing Hpc
computing BLA
40
100
computing Hpc
computing BLA
40
100


C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2559: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2560: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
100% |########################################################################|


computing Hpc
computing BLA


In [28]:
plt.figure()
plt.subplot(1,3,1)
plt.imshow(stru_corr['Hpc'][21000,:,:])
plt.clim(-1,1)
plt.subplot(1,3,2)
plt.imshow(np.abs(stru_corr['Hpc'][21000,:,:]))
plt.clim(-1,1)
plt.subplot(1,3,3)
plt.colorbar()

NameError: name 'stru_corr' is not defined

In [10]:
for i in range(0,binned.shape[1],step):
    plt.clf()
    corr = np.corrcoef(binned[:,i:i+winSize])
    corr[np.isnan(corr)] = 0
    sns.heatmap(corr,square = True,vmin = 0,vmax = 0.1,cmap = 'viridis')
    plt.title(i)
    plt.draw()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

KeyboardInterrupt: 

In [283]:

x = np.arange(0,25,0.001)
y = np.sin(x)
y_g = scipy.signal.convolve(y,g,'same')

plt.plot(x,y)
plt.plot(x,y_g)

In [317]:
g = scipy.signal.windows.gaussian(50000,1,sym=True)
np.sum(g)

2.5066282612190953

In [318]:
plt.plot(g)